* Variable	Definition	Key
* survival	Survival	0 = No, 1 = Yes
* pclass	Ticket class	1 = 1st, 2 = 2nd, 3 = 3rd
    * 1st = Upper
    * 2nd = Middle
    * 3rd = Lower
* sex	Sex	
* Age	Age in years	
* sibsp	# of siblings(兄弟) / spouses（配偶者） aboard the Titanic	
* parch	# of parents（親） / children（子供） aboard the Titanic	
* ticket	Ticket number	
* fare	Passenger fare	
* cabin	Cabin number	
* embarked	Port of Embarkation	C = Cherbourg, Q = Queenstown, S = Southampton

In [7]:
#テスト
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import lightgbm as lgb
from sklearn.model_selection import 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


import os

In [25]:
train = pd.read_csv("../Documents/kaggle/kaggle_team/titanic_data/train.csv")
test = pd.read_csv("../Documents/kaggle/kaggle_team/titanic_data/test.csv")

### データの確認

In [26]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [27]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [28]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [ ]:
### 

In [29]:
train.drop(["PassengerId", "Name", "Ticket"], axis=1, inplace=True)
train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,7.2500,NaN,S
1,1,1,female,38.0,1,0,71.2833,C85,C
2,1,3,female,26.0,0,0,7.9250,NaN,S
3,1,1,female,35.0,1,0,53.1000,C123,S
4,0,3,male,35.0,0,0,8.0500,NaN,S


In [30]:
test.drop(["PassengerId", "Name", "Ticket"], axis=1, inplace=True)
test.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,3,male,34.5,0,0,7.8292,NaN,Q
1,3,female,47.0,1,0,7.0000,NaN,S
2,2,male,62.0,0,0,9.6875,NaN,Q
3,3,male,27.0,0,0,8.6625,NaN,S
4,3,female,22.0,1,1,12.2875,NaN,S


In [31]:
label = train["Survived"]
train.drop(["Survived"], axis=1, inplace=True)

In [32]:
train.shape
train.dtypes

Pclass        int64
Sex          object
Age         float64
SibSp         int64
Parch         int64
Fare        float64
Cabin        object
Embarked     object
dtype: object

In [33]:
cat_feat = ["Pclass", "Sex", "Cabin", "Embarked"]
for i in cat_feat :
    train[i] = train[i].astype("category")
    test[i] = test[i].astype("category")

In [34]:
train.shape
train.dtypes

Pclass      category
Sex         category
Age          float64
SibSp          int64
Parch          int64
Fare         float64
Cabin       category
Embarked    category
dtype: object

In [35]:
X_train, X_valid, y_train, y_valid = train_test_split(train, label)

In [36]:
lgb_train = lgb.Dataset(X_train, y_train, categorical_feature=cat_feat)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train, categorical_feature=cat_feat)

params = {
    "objective" : "binary",
    "metric" : "binary_logloss",
    "early_stopping_round" : 10
}

In [37]:
model = lgb.train(params, lgb_train, valid_sets=[lgb_train, lgb_eval])

[LightGBM] [Info] Number of positive: 252, number of negative: 416
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000600 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 204
[LightGBM] [Info] Number of data points in the train set: 668, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.377246 -> initscore=-0.501256
[LightGBM] [Info] Start training from score -0.501256
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1]	training's binary_logloss: 0.612063	valid_1's binary_logloss: 0.638515
Training until validation scores don't improve for 10 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2]	training's binary_logloss: 0.57091	valid_1's binary_logloss: 0.609782
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3]	training

/Users/terasawayasuto/opt/anaconda3/lib/python3.9/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/Users/terasawayasuto/opt/anaconda3/lib/python3.9/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/Users/terasawayasuto/opt/anaconda3/lib/python3.9/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


In [38]:
val_pred = model.predict(X_valid, num_iteration=model.best_iteration)
val_pred

array([0.09878459, 0.42802834, 0.88495623, 0.20164183, 0.86725797,
       0.11933778, 0.21675693, 0.18203594, 0.34975657, 0.21682679,
       0.55863611, 0.17754288, 0.76114974, 0.51536136, 0.06135876,
       0.34606989, 0.4059464 , 0.11525634, 0.12937778, 0.41514966,
       0.53782943, 0.12941478, 0.17270792, 0.10268048, 0.42857302,
       0.07915334, 0.1451834 , 0.89003233, 0.81037893, 0.20494751,
       0.85770649, 0.07915334, 0.45543072, 0.17959075, 0.36215727,
       0.18697355, 0.37710765, 0.31666455, 0.35958619, 0.17270792,
       0.11819621, 0.3910017 , 0.08401736, 0.52343774, 0.08548799,
       0.11078324, 0.2396088 , 0.11424783, 0.12319963, 0.86060984,
       0.24696965, 0.23425372, 0.22039783, 0.12217562, 0.87500132,
       0.12319963, 0.0949703 , 0.52024866, 0.45933652, 0.1424984 ,
       0.09514375, 0.16965012, 0.17071166, 0.87500132, 0.16207523,
       0.17013105, 0.3910017 , 0.08401736, 0.5743739 , 0.81037893,
       0.09353529, 0.89178197, 0.09647082, 0.33839014, 0.22106

In [39]:
val_pred = np.where(val_pred < 0.5, 0, 1)
acc = accuracy_score(y_valid, val_pred)
acc

0.7668161434977578

In [40]:
test_pred = model.predict(test, num_iteration=model.best_iteration)
test_pred = np.where(test_pred < 0.5, 0, 1)
test_pred

array([0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [42]:
sub = pd.read_csv("../Documents/kaggle/kaggle_team/titanic_data/gender_submission.csv")

In [43]:
sub.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [44]:
sub["Survived"] = test_pred
sub

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [45]:
sub.to_csv("../Documents/kaggle/kaggle_team/titanic_data/submission/submission1.csv", index=False)